In [34]:
#pip install -U git+https://github.com/QuantFreedom1022/quantfreedom@dev

In [35]:
#pip install vectorbt

In [36]:
#installation of Yahoo Finance 
#!pip install yfinance

In [5]:
### Importing QuantFreedom and all requirements for strategy creation

import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

#from quantfreedom.nax_qf.eval_nax import eval_is_above

import quantfreedom as qf
#import vectorbt as vbt
import yfinance as yf
import numpy as np
import pandas as pd
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash

import plotly.express as px
import plotly.graph_objects as go

import talib
from talib.abstract import Function


# ^ - importing in next cell
# ^^^ from quantfreedom.backtester.indicators.talib_ind import from_talib, talib_func_list_website_link, talib_ind_info
# ^^^ 
"""*** from quantfreedom.evaluators.evaluators import eval_is_below, eval_is_above, combine_evals, eval_is_closed_above, eval_is_closed_below"""
from quantfreedom._typing import (
    pdFrame, pdSeries)


#!!!!!!!!!! ############################ if I choose kernel Python 3.10, I got error on 8 cell. If I choose qfFree, I got error on cell 8
# it's start to happen when I'm uncomment this row: from quantfreedom.backtester.evaluators.evaluators import eval_is_below, eval_is_above, combine_evals
"""
Trying to fix it: 1) Delete and install extensions again: jupyter, python, Pylance, Python Environment Manager DONE! Not helped.

2) Removing all venv by a) jupyter kernelspec list b) than: jupyter kernelspec remove <kernel_name>

3) Uninstall Python in Windows and install again
"""


'\nTrying to fix it: 1) Delete and install extensions again: jupyter, python, Pylance, Python Environment Manager DONE! Not helped.\n\n2) Removing all venv by a) jupyter kernelspec list b) than: jupyter kernelspec remove <kernel_name>\n\n3) Uninstall Python in Windows and install again\n'

In [2]:
####### Importing all QuantFreedom backtesting stuff #######

# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

# ^ - already imported in previous cell
# ^^^ import plotly.express as px
# ^^^ import plotly.graph_objects as go

# ^ - already imported in previous cell
# ^^^ import pandas as pd
# ^^^ import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.nb.simulate import backtest_df_array_only_nb
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.indicators.talib_ind import from_talib
"""*from quantfreedom.evaluators.evaluators import eval_is_below, eval_is_above, combine_evals, eval_is_closed_above"""
"""* , eval_is_closed_above, eval_is_closed_below"""
""" from quantfreedom.nax_qf.eval_nax import eval_is_closed_above, eval_is_closed_below """

"""from quantfreedom.nax_qf.eval_nax import eval_is_closed_above, eval_is_closed_below"""


# np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

# pd.options.display.float_format = '{:,.2f}'.format



'from quantfreedom.nax_qf.eval_nax import eval_is_closed_above, eval_is_closed_below'

In [6]:
"""


# Fetch historical data from Yahoo Finance

# determine variable 'symbol' for fetching many tickets simultaneously (at same time)
# symbol = [
#     'BTC-USD', 'ETH-USD', 'XRP-USD', 'BCH-USD', 'LTC-USD'#,
# ]
# """    'BNB-USD', 'EOS-USD', 'XLM-USD', 'XMR-USD', 'ADA-USD',
#     'QBIT-USD', 'TSLA', 'AMZN', 'AAPL', 'DGTX-USD', 'BTG-USD', 
#     'LINK-USD', 'DOGE-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD',
#     'TON-USD', 'OKB-USD', 'GLD', 'GIS', 'PG', 'MRK', 'AMT', 'BMY',
#     'CVX', 'S', 'CVS', 'HD', 'PUK'
# #]
# """


symbol = ['BTC-USD', 'ETH-USD']

# "Attach" dataset prices (dataframe) to variable ohlcv and at the same time download dataset from Yahoo Finance 
ohlcv = yf.download(symbol, interval="5m", period="60d")

# "Attach" only closing prices from dataset to variable 'price'
prices = ohlcv['Close']

# "Attach" only High prices from dataset to variable 'high_price'
high_price = ohlcv['High']

# Same for low
low_price = ohlcv['Low']


"""
"""

[*********************100%***********************]  2 of 2 completed


'\n'

In [7]:
ohlcv

Adj Close              Close               High   
                            BTC-USD  ETH-USD   BTC-USD  ETH-USD   BTC-USD   
Datetime                                                                    
2023-02-08 00:00:00+00:00 23,267.40 1,672.07 23,267.40 1,672.07 23,267.40  \
2023-02-08 00:05:00+00:00 23,237.73 1,669.60 23,237.73 1,669.60 23,264.89   
2023-02-08 00:10:00+00:00 23,243.34 1,669.36 23,243.34 1,669.36 23,251.24   
2023-02-08 00:15:00+00:00 23,259.08 1,670.11 23,259.08 1,670.11 23,259.08   
2023-02-08 00:20:00+00:00 23,256.87 1,670.75 23,256.87 1,670.75 23,258.82   
...                             ...      ...       ...      ...       ...   
2023-04-08 01:35:00+00:00 27,933.93 1,860.54 27,933.93 1,860.54 27,940.33   
2023-04-08 01:40:00+00:00 27,936.17 1,861.21 27,936.17 1,861.21 27,938.36   
2023-04-08 01:45:00+00:00 27,932.93 1,861.00 27,932.93 1,861.00 27,936.85   
2023-04-08 01:50:00+00:00 27,933.14 1,861.18 27,933.14 1,861.18 27,933.14   
2023-04-08 01:55:00+00:00 27,926.02 1,860.68 27,926.02 1,860.68 27,926.02   

                                         Low               Open            
                           ETH-USD   BTC-USD  ETH-USD   BTC-USD  ETH-USD   
Datetime                                                                   
2023-02-08 00:00:00+00:00 1,672.07 23,259.63 1,671.60 23,263.42 1,671.91  \
2023-02-08 00:05:00+00:00 1,671.83 23,234.14 1,669.60 23,263.15 1,671.83   
2023-02-08 00:10:00+00:00 1,669.83 23,243.34 1,669.36 23,247.46 1,669.39   
2023-02-08 00:15:00+00:00 1,670.11 23,245.96 1,669.38 23,245.96 1,669.51   
2023-02-08 00:20:00+00:00 1,670.75 23,254.02 1,669.72 23,258.65 1,670.14   
...                            ...       ...      ...       ...      ...   
2023-04-08 01:35:00+00:00 1,861.43 27,929.37 1,860.36 27,929.37 1,860.36   
2023-04-08 01:40:00+00:00 1,861.31 27,934.39 1,860.97 27,934.39 1,860.97   
2023-04-08 01:45:00+00:00 1,861.58 27,932.93 1,861.00 27,936.85 1,861.36   
2023-04-08 01:50:00+00:00 1,861.18 27,916.23 1,859.03 27,929.95 1,860.20   
2023-04-08 01:55:00+00:00 1,860.68 27,926.02 1,860.68 27,926.02 1,860.68   

                             Volume            
                            BTC-USD   ETH-USD  
Datetime                                       
2023-02-08 00:00:00+00:00         0         0  
2023-02-08 00:05:00+00:00  16879616         0  
2023-02-08 00:10:00+00:00  22849536   7335936  
2023-02-08 00:15:00+00:00         0         0  
2023-02-08 00:20:00+00:00  14008320  16365056  
...                             ...       ...  
2023-04-08 01:35:00+00:00  14644224   7598592  
2023-04-08 01:40:00+00:00    982016    584704  
2023-04-08 01:45:00+00:00   3872768   1290752  
2023-04-08 01:50:00+00:00   1884160   3306496  
2023-04-08 01:55:00+00:00         0         0  

[17016 rows x 12 columns]

In [14]:
ohlcv = pd.read_hdf(
    '/coding/QuantFreedom/tests/data/prices.hd5', index_col='time')

In [11]:
#Renaming columns for TA-Lib
ohlcv.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Adj Close': 'close', 'Volume': 'volume'}, inplace=True)

In [12]:
ohlcv.head(5)

symbol                BTCUSDT                               XRPUSDT             
candle_info              open      high       low     close    open high  low   
open_time                                                                       
2022-04-01 00:00:00 45,511.00 45,639.00 45,414.50 45,600.50    0.81 0.83 0.81  \
2022-04-01 00:30:00 45,600.50 45,630.00 45,340.00 45,540.00    0.83 0.83 0.82   
2022-04-01 01:00:00 45,540.00 45,540.50 45,112.00 45,139.50    0.83 0.83 0.82   
2022-04-01 01:30:00 45,139.50 45,248.50 44,801.00 44,919.50    0.82 0.83 0.82   
2022-04-01 02:00:00 44,919.50 45,040.50 44,529.00 44,649.50    0.82 0.83 0.81   

symbol                     
candle_info         close  
open_time                  
2022-04-01 00:00:00  0.83  
2022-04-01 00:30:00  0.83  
2022-04-01 01:00:00  0.82  
2022-04-01 01:30:00  0.82  
2022-04-01 02:00:00  0.81

In [43]:
prices.head(5)

Datetime
2023-02-07 00:00:00+00:00   22,764.50
2023-02-07 00:05:00+00:00   22,786.85
2023-02-07 00:10:00+00:00   22,804.49
2023-02-07 00:15:00+00:00   22,823.20
2023-02-07 00:20:00+00:00   22,805.57
Name: Close, dtype: float64

In [18]:
 
rsi_ind = from_talib(
    func_name='rsi',
    prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=30,
)


 
# talib.abstract.RSI(prices)


AttributeError: 'Index' object has no attribute 'levels'

In [ ]:
# Define Divergence
def divergence(
    # using price set as starting point to calculate divergence
    price_set,
    # using last 14 bars/days period window for RSI indicator. This is just a template and can be changed when we will execute .run function in next stage.
    rsi_window,
    # using last 1 bar/day period window for fast Moving Average indicator. This is just a template and can be changed when we will execute .run function in next stage.
    fast_ma_window,

    # using last 50 bars/days period window for fast Moving Average indicator.  This is just a template and can be changed when we will execute .run function in next stage.
        slow_ma_window,):

    slow_ma = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=slow_ma_window,
    )
    fast_ma = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=fast_ma_window,
    )
    rsi_indicator = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=rsi_window,
    ).squeeze()

# Calculate the Divergence indicator. Arithmetics behind indicators is black box for me and consist of pure mathematics
    bullish_divergence = ((price_set.diff() < 0) & (rsi_indicator.diff() > 0)).to_numpy()

    bearish_divergence = ((price_set.diff() < 0) &
                          (rsi_indicator.diff() < 0)).to_numpy()

    # Determine UpTrend (fast MA above slow MA)
    trend_up = eval_is_closed_above(
        indicator_data=fast_ma,
        want_to_evaluate=slow_ma.squeeze(),
    )
    trend_down = eval_is_closed_below(
        indicator_data=fast_ma,
        want_to_evaluate=slow_ma.squeeze(),
    )

    # Convert everything to Numpy array
    trend_up = trend_up.to_numpy().flatten()
    trend_down = trend_down.to_numpy().flatten()

    # Combine all previous indicators into the one
    # We need that if bullish_divergence (trend_up), True PLUS fast MA above slow MA, then set 1, everything else - 0. If bearish_divergence True, PLUS fast MA above slow MA (trend_down), then set -1
    indicator_buy = np.logical_and(bullish_divergence == True,trend_up == True)
    indicator_buy = np.where(indicator_buy == True, 1, 0)
    indicator_sell = np.logical_and(bearish_divergence == True,trend_down == True)
    indicator = np.where(indicator_sell == True, -1, indicator_buy)


    return indicator


In [ ]:
print(type(divergence))


price_set = ohlcv['close']
rsi_window = 14
fast_ma_window = 1
slow_ma_window = 50

# Call the function and print the value of indicator
indicator = divergence(price_set, rsi_window, fast_ma_window, slow_ma_window)
print(indicator)

In [ ]:
# Just for testing purposes - can be deleten

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=14,
)

rsi_eval = eval_is_closed_below(
    rsi_ind,
    np.arange(20),
)


rsi_eval.head(55)
 


In [ ]:
# Define backtester

strat_array, settings_array = backtest_df_array_only(
    open_prices=ohlcv.open.values,
    high_prices=ohlcv.high.values,
    low_prices=ohlcv.low.values,
    close_prices=ohlcv.close.values,
    entries=rsi_eval.values,
    equity=1000.,
    fee_pct=.02,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(1,6.1,1),
    risk_rewards=np.arange(1,10.1,1),
    size_pct=1.,
    gains_pct_filter=100,
    # sl_pcts=np.arange(1,6,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(1,6.1,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,5.1,1),
    tsl_when_pct_from_avg_entry=np.arange(1,5.1,1),
)

In [ ]:
# Evaluators and backtesting


In [ ]:
atr_ind = from_talib(
    func_name='atr',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=1,
)

talib.SMA(atr_ind.values.flatten(), timeperiod=50)

In [ ]:
"""
pf = vbt.Portfolio.from_signals(ohlcv, divergence(indicator_buy), divergence(indicator_sell))

# Output
print(pf.total_return())
print(pf.stats())
"""

In [ ]:
divergence(
    price_set=ohlcv.close,
    rsi_window=15,
    fast_ma_window=10,
    slow_ma_window=40,
)

In [ ]:
atr_ind = from_talib(
    func_name='atr',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=1,
)

In [ ]:
atr_ind

In [ ]:
talib.SMA(atr_ind.values.flatten(), timeperiod=50)


In [ ]:
(1000 / ohlcv.close.values.flatten())

In [ ]:
talib.SMA(atr_ind.values.flatten(), timeperiod=50) * (1000 / ohlcv.close.values.flatten())

In [ ]:
fast_ma = vbt.MA.run(price_set, fast_ma_window, short_name='fast')
#printing tpe of bearish_divergence 
print("printing tpe of bearish_divergence dataset:")
print(type(bearish_divergence))
# Get slow Moving Average (MA) of the prices

atr = vbt.ATR.run(high_price, low_price, price, atr_window)
#atr = atr.atr.to_numpy()

atr = atr.atr.to_numpy()
##print(type(atr))

# Calculate adaptive/dynamically changing value based on ATR and current price 
atr_results = atr * atr_scaler / price #* rsi_window

slow_ma = vbt.MA.run(atr_results, slow_ma_window, short_name='slow')
print("printing tpe of slow_ma dataset:")
print(type(slow_ma))

In [ ]:
fig = go.Figure()
fig.add_scatter(
    x=atr_ind.index.to_list(),
    y=ohlcv.close.values.flatten(),
    mode='lines',
    marker=dict(color='red'),
    name='ATR'

)
fig.add_scatter(
    x=rsi_ind.index.to_list(),
    y=talib.SMA(atr_ind.values.flatten(), timeperiod=100) * 1000 / ohlcv.close.values.flatten(),
    mode='lines',
    marker=dict(color='blue'),
    name='SMA'
)
fig.show()


In [ ]:
atr_ind.values.flatten()

In [ ]:
atr_ind

In [ ]:
atr_ma = from_talib(
    func_name='MA',
    df_prices=None,
    cart_product=False,
    combos=False,
    timeperiod=50,
    price=atr_ind.values.flatten()
)

In [ ]:
talib_ind_info('stoch')

In [ ]:
close = np.random.random(100)
open = np.random.random(100)
high = np.random.random(100)

In [ ]:
Function('sma')(np.random.random(10000), 50)

In [ ]:
_, hey = talib.STOCH(close, open, high, *(10,15))
hey